In [37]:
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
# Import dataset and bounding boxes of countries. This is used in world map
dataset = 'TILL6022_Emission_Dataset.csv'
df = pd.read_csv(dataset, delimiter=',', encoding='ISO-8859-1')
country_bb = pd.read_csv('country_bb.csv', delimiter=',', encoding='ISO-8859-1')
transport_sectors = ['Total', 'International Aviation', 'Domestic Aviation', 'Ground Transport']

In [38]:
# Select certain parameters, that will otherwise be selected by streamlit
given_sector = 'Total'
given_country = 'China'
given_time = 'Week'


In [39]:
# Date modifications to filter data
if given_time == 'Day':
    date_filt = 'D'
elif given_time == 'Week':
    date_filt = 'W-MON'
elif given_time == 'Month':
    date_filt = 'MS'
elif given_time == 'Quartile':
    date_filt = 'QS'

In [48]:
# Group data on dates to filter data and smooth data regarding to dates
df.date = pd.to_datetime(df.date)
df_filt = df.groupby(['country','sector']).resample(date_filt,on='date').sum()    #D for Day, MS for month, QS to quarter, Y for year
df_filt = df_filt.reset_index()
box_pts = 4
box = np.ones(box_pts)/box_pts
df_filt['co2_smooth'] = np.convolve(df_filt.co2, box, mode='same')

C:\Users\timtr\AppData\Local\Temp\ipykernel_23480\1784955690.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\timtr\AppData\Local\Temp\ipykernel_23480\1784955690.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\timtr\AppData\Local\Temp\ipykernel_23480\1784955690.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\timtr\AppData\Local\Temp\ipykernel_23480\1784955690.py:3: FutureWarning:

Th

In [41]:
# By filtering data, the first and last data points will be removed, since these are not smoothed correctly
df_filt = df_filt.set_index('date')
date_min = df_filt.index.min()
date_max = df_filt.index.max()
index_min = df_filt.index == date_min
index_max = df_filt.index == date_max
df_filt = df_filt.reset_index()
df_filt.date = np.datetime_as_string(df_filt.date, unit='D')
index_min_1 = pd.Series(df_filt.index[index_min])
index_min_2 = pd.Series(df_filt.index[index_min]+1)
index_max_1 = pd.Series(df_filt.index[index_max])
index_max_2 = pd.Series(df_filt.index[index_max]-1)
index_series = pd.Index(pd.concat([index_max_1,index_max_2,index_min_1,index_min_2], ignore_index=True))
df_filt.drop(index_series, axis=0, inplace= True)

In [42]:
# Zoom possibilities to countries are introduced for the world map. When selecting a country, you zoom
zoom_factor = 3
if (given_country == 'WORLD' or given_country == 'ROW' or given_country == 'EU27 & UK'):
    latrange = [-90,90]
    lonrange = [-180,180]
elif given_country == 'US':
    bb_country = 'United States'
    given_country_bb = country_bb[country_bb.country == bb_country]
    latrange = [float(given_country_bb['latmin'])-zoom_factor,float(given_country_bb['latmax'])+zoom_factor]
    lonrange = [float(given_country_bb['lonmin'])-zoom_factor*2,float(given_country_bb['lonmax'])+zoom_factor*2]
elif given_country == 'UK':
    bb_country = 'United Kingdom'
    given_country_bb = country_bb[country_bb.country == bb_country]
    latrange = [float(given_country_bb['latmin'])-zoom_factor,float(given_country_bb['latmax'])+zoom_factor]
    lonrange = [float(given_country_bb['lonmin'])-zoom_factor*2,float(given_country_bb['lonmax'])+zoom_factor*2]
else:
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin'])-zoom_factor,float(given_country_bb['latmax'])+zoom_factor]
    lonrange = [float(given_country_bb['lonmin'])-zoom_factor*5,float(given_country_bb['lonmax'])+zoom_factor*5]
data_sector = df_filt[df_filt.sector == given_sector]
if given_country == 'WORLD':
    leg_min = float(data_sector[data_sector.country != 'WORLD']['co2'].min())
    leg_max = float(data_sector[data_sector.country != 'WORLD']['co2'].max())
elif len(df_filt[df_filt.sector == given_sector].country.unique())>1:
    leg_min = float(data_sector[data_sector.country == given_country]['co2'].min())
    leg_max = float(data_sector[data_sector.country == given_country]['co2'].max())


Write something about the zoom function and the implemenetation of geojson files and why they are not used.

In [43]:
# Creating world map 
fig = px.choropleth(df_filt[df_filt.sector == given_sector], 
                    locations= 'country',
                    locationmode= "country names",
                    color='co2',
                    range_color= [leg_min, leg_max],
                    animation_frame= 'date'
                    )
fig.update_geos(lataxis_range=latrange, lonaxis_range=lonrange)  
fig.update_layout(
    title_text = "CO2 emmissions transport sector",
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection_type = "equirectangular",
    ),
    height=700,
    width  = 1200
)
fig.show()

In [44]:
# Filter dataset on Transport sectors: 'International Shipping' or 'International Aviation' or 'Domestic Aviation' or 'Ground Transport'
# Transport_data is used to filter specific data instead of having to use a large command several times.
transport_data =  df_filt[df_filt.sector.isin(transport_sectors)]

# Reset index for optimal dataset and split date into years. This is necessary to plot multiple years in a single plot
transport_data[["year", "month", "day"]] = transport_data["date"].str.split("-", expand = True)
transport_data['same_year'] = '2019' + '-' + transport_data['month'].astype(str) + '-' + transport_data['day'] # Same year is created to visualize in same plot
Years = transport_data.year.unique()
Countries = transport_data.country.unique()

# Remove countries that are not used
Countries = np.delete(Countries, np.where(Countries == ['ROW']))
Countries = np.delete(Countries, np.where(Countries == ['WORLD']))

In [45]:
# Creating empty lists 
Delta_Country = []
Date_country = []
Date_country_max = []
Percentage_Delta = []
Delta_Country_Do = []
Date_country_Do = []
Date_country_max_Do = []
Percentage_Delta_Do = []
Delta_Country_Trans = []
Date_country_Trans = []
Date_country_max_Trans = []
Percentage_Delta_Trans = []
Delta_Country_Int = []
Date_country_Int = []
Date_country_max_Int = []
Percentage_Delta_Int = []

In [46]:

fig_transport = make_subplots(rows = 2, cols = 2,
                    subplot_titles = (transport_sectors))

for k in Countries:
        for i in Years:
                trans_tot = transport_data[(transport_data.country == k)].groupby(
                        ['date'],as_index = False).agg({'country':'first','co2':'sum',
                        'co2_smooth':'sum','year':'first','month':'first','day':'first','same_year':'first'})
                trans_int_av = transport_data[(transport_data.country == k)
                        & (transport_data.sector == 'International Aviation')]
                trans_do_av = transport_data[(transport_data.country == k)
                        & (transport_data.sector == 'Domestic Aviation')]
                trans_gr_tr = transport_data[(transport_data.country == k)
                        & (transport_data.sector == 'Ground Transport')]
                lop = fig_transport.add_traces(
                        [
                                go.Scatter(x=list(trans_tot[(trans_tot.year == i)].same_year),
                                        y=list(trans_tot[(trans_tot.year == i)].co2),
                                        name= k + ' ' + i + ' ' + 'Total',
                                        visible = True,
                                        legendgroup = 1),
                                go.Scatter(x=list(trans_int_av[(trans_int_av.year == i)].same_year),
                                        y=list(trans_int_av[(trans_int_av.year == i)].co2),
                                        name= k + ' ' + i + ' ' + 'International Aviation',
                                        visible = True,
                                        legendgroup = 2),
                                go.Scatter(x=list(trans_do_av[(trans_do_av.year == i)].same_year),
                                        y=list(trans_do_av[(trans_do_av.year == i)].co2),
                                        name= k + ' ' + i + ' ' + 'Domestic Aviation',
                                        visible = True,
                                        legendgroup = 3),
                                go.Scatter(x=list(trans_gr_tr[(trans_gr_tr.year == i)].same_year),
                                        y=list(trans_gr_tr[(trans_gr_tr.year == i)].co2),
                                        name= k + ' ' + i + ' ' + 'Ground Transport',
                                        visible = True,
                                        legendgroup = 4)],
                                rows=[1,2,1,2], cols=[1,1,2,2]
                )
        
        # Total sectors deviation
        Delta_Total = trans_tot[(trans_tot.country == k)]
        diff_list = []
        for d in range(len(Delta_Total)-1):
                Diff = abs(Delta_Total['co2_smooth'].iloc[d] - Delta_Total['co2_smooth'].iloc[d+1])
                Co2Value = Delta_Total['co2_smooth'].iloc[d-1]
                diff_list.append(Diff)
                Date_country.append(Delta_Total.iloc[d]['date'])
        Max_Diff = max(diff_list)
        percent_Delta = ("%g" % round((Max_Diff/Co2Value)*100,1))
        index_maxvalue = diff_list.index(Max_Diff)
        maxDATE = Date_country[index_maxvalue]
        Delta_Country.append(Max_Diff)
        Date_country_max.append(maxDATE)
        Percentage_Delta.append(percent_Delta)

        # Domestic sector 
        Do_Av = trans_do_av[(trans_do_av.country == k)]
        diff_list_Do = []
        for d in range(len(Do_Av)-1):
                Diff = abs(Do_Av['co2_smooth'].iloc[d] - Do_Av['co2_smooth'].iloc[d+1])
                Co2Value_Do = Do_Av['co2_smooth'].iloc[d-1]
                diff_list_Do.append(Diff)
                Date_country_Do.append(Do_Av.iloc[d]['date'])
        Max_Diff_Do = max(diff_list_Do)
        percent_Delta_Do = round((Max_Diff_Do/Co2Value_Do)*100,1)
        index_maxvalue_Do = diff_list_Do.index(Max_Diff_Do)
        maxDATE_Do = Date_country_Do[index_maxvalue_Do]
        Delta_Country_Do.append(Max_Diff_Do)
        Date_country_max_Do.append(maxDATE_Do)
        Percentage_Delta_Do.append(percent_Delta_Do)
        
        Gr_Trans = trans_gr_tr[(trans_gr_tr.country == k)]
        diff_list_Trans = []
        for d in range(len(Gr_Trans)-1):
                Diff = abs(Gr_Trans['co2_smooth'].iloc[d] - Gr_Trans['co2_smooth'].iloc[d+1])
                Co2Value_Trans = Gr_Trans['co2_smooth'].iloc[d-1]
                diff_list_Trans.append(Diff)
                Date_country_Trans.append(Gr_Trans.iloc[d]['date'])
        Max_Diff_Trans = max(diff_list_Trans)
        Percent_Delta_Trans = round((Max_Diff_Trans/Co2Value_Trans)*100, 1)
        # print('This is the max diff:' , Max_Diff_Trans, k)
        Delta_Country_Trans.append(Max_Diff_Trans)
        index_maxvalue_Trans = diff_list_Trans.index(Max_Diff_Trans)
        maxDATE_Trans = Date_country_Trans[index_maxvalue_Trans]
        Date_country_max_Trans.append(maxDATE_Trans)
        Percentage_Delta_Trans.append(Percent_Delta_Trans)
        # print(Delta_Country)



        # for i in Years:
        Int_Av = trans_int_av[(trans_int_av.country == k)]
        diff_list_Int = []
        for d in range(len(Int_Av)-1):
                Diff = abs(Int_Av['co2_smooth'].iloc[d] - Int_Av['co2_smooth'].iloc[d+1])
                Co2Value_Int = Int_Av['co2_smooth'].iloc[d-1]
                diff_list_Int.append(Diff)
                Date_country_Int.append(Int_Av.iloc[d]['date'])
        Max_Diff_Int = max(diff_list_Int)
        Percent_Delta_Int = round((Max_Diff_Int/Co2Value_Int)*100, 1)
        Delta_Country_Int.append(Max_Diff_Int)
        index_maxvalue_Int = diff_list_Int.index(Max_Diff_Int)
        maxDATE_Int = Date_country_Int[index_maxvalue_Int]
        Date_country_max_Int.append(maxDATE_Int)
        Percentage_Delta_Int.append(Percent_Delta_Int)

list_updatemenus = []

for n, country in enumerate(Countries):
    visible = 4*[False] * len(Years) * len(Countries)
    try:
        for x in range(4*len(Years)):
            visible[4*len(Years)*n + x] = True
    except IndexError:
        continue
    temp_dict = dict(label = str(country),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': ('CO2 emissions comparison 2019 versus 2020 by sector in %s'% country)}])
    list_updatemenus.append(temp_dict)

fig_transport.update_layout(updatemenus=list([dict(buttons= list_updatemenus)]))

fig_transport.update_layout(
    updatemenus=[
        dict(
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=1.04,
            yanchor="top"
        ),
    ]
)

# add height, title, labels etc. to figure

fig_transport.update_layout(
    height = 1600,
    title_text = 'CO2 emissions comparison 2019 until 2022',
    
    xaxis1_title = 'Date',
    xaxis2_title = 'Date',
    xaxis3_title = 'Date',
    xaxis4_title = 'Date',
    yaxis1_title = 'CO2 emissions x 1000 (ppm)',
    yaxis2_title = 'CO2 emissions x 1000 (ppm)',
    yaxis3_title = 'CO2 emissions x 1000 (ppm)',
    yaxis4_title = 'CO2 emissions x 1000 (ppm)',
    
    xaxis=dict(tickformat="%d-%B"),
                 xaxis2=dict(tickformat="%d-%B"),
                 xaxis3=dict(tickformat="%d-%B"),
                 xaxis4=dict(tickformat="%d-%B"),
    hovermode = 'x unified',
    legend_tracegroupgap = 340,
    font=dict(       
        size=14
    )
    )

# show figure
fig_transport.show()


In [47]:
DeltaTable = pd.DataFrame(
    {'Country': Countries,
     'Biggest Delta': Delta_Country,
     'Percentage biggest Delta [%]': Percentage_Delta,
     'Date biggest Delta': Date_country_max,
     'Biggest Ground Transportation Delta': Delta_Country_Trans,
     'Percentage biggest Ground Transportation Delta [%]': Percentage_Delta_Trans,
     'Date biggest Ground Transportation Delta': Date_country_max_Trans,
     'Biggest Domestic Aviation Delta': Delta_Country_Do,
     'Percentage biggest Domestic Aviation Delta [%]': Percentage_Delta_Do,
     'Date biggest Domestic Aviation Delta': Date_country_max_Do,
     'Biggest International Aviation Delta': Delta_Country_Int,
     'Percentage biggest International Aviation Delta [%]': Percentage_Delta_Int,
     'Date biggest International Aviation Delta': Date_country_max_Int
    })


DeltaTable.style.background_gradient(cmap='Blues')